In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
from sklearn.metrics import matthews_corrcoef

y_true = [1, 1, 1, 0]
y_pred = [1, 0, 1, 1]

score = matthews_corrcoef(y_true, y_pred)

score

-0.3333333333333333

In [6]:
score = matthews_corrcoef(y_true, y_true)

score

1.0

In [7]:
y_pred = [0, 0, 0, 1]

score = matthews_corrcoef(y_true, y_pred)

score

-1.0

In [8]:
import os 

path = 'data/eopatches/train'
patches = os.listdir(path)
len(patches)

100

In [15]:
from src.utils import generate_mask_with_mean_ndvi

patch = patches[0]

mask = generate_mask_with_mean_ndvi(path, patch)

mask.shape, mask.dtype, mask.max(), mask.min()

((2000, 2000), dtype('bool'), True, False)

In [22]:
from src.utils import get_npy

target = get_npy(path, patch, 'mask_timeless/CULTIVATED')[...,0]

target.shape, target.dtype, target.max(), target.min()

((2000, 2000), dtype('bool'), True, False)

In [23]:
import matplotlib.pyplot as plt 

fig = plt.figure()
ax = plt.subplot(1, 2, 1)
ax.imshow(target)
ax.axis('off')
ax = plt.subplot(1, 2, 2)
ax.imshow(mask)
ax.axis('off')
plt.tight_layout()
plt.plot()

[]